<a href="https://colab.research.google.com/github/imdeja/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/module4-Hyperparameter-Tuning/Jake_Dennis_LS_DS_434_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [0]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam, SGD

In [2]:
df = pd.read_csv('https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

print(df.shape)
df.head()

(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [4]:
df = df.drop(columns='customerID')
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'],errors='coerce')
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [0]:
encode = df.describe(exclude='number').columns.tolist()
encoder = OrdinalEncoder()
encoded = encoder.fit_transform(df[encode])

In [7]:
df_ordinal = pd.DataFrame(data=encoded, columns=encode)
df_ordinal.head()

,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0
3,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0


In [8]:
df = df.drop(columns=encode)
df.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
0,0,1,29.85,29.85
1,0,34,56.95,1889.50
2,0,2,53.85,108.15
3,0,45,42.30,1840.75
4,0,2,70.70,151.65


In [9]:
df = pd.concat([df, df_ordinal], axis=1)
df.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
0,0,1,29.85,29.85,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0
1,0,34,56.95,1889.50,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0
2,0,2,53.85,108.15,1.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0
3,0,45,42.30,1840.75,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,2,70.70,151.65,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0


In [10]:
df.isnull().sum()

SeniorCitizen        0
tenure               0
MonthlyCharges       0
TotalCharges        11
gender               0
Partner              0
Dependents           0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
Churn                0
dtype: int64

In [11]:
df = df.fillna(df['TotalCharges'].mean())
df.isnull().sum()

SeniorCitizen       0
tenure              0
MonthlyCharges      0
TotalCharges        0
gender              0
Partner             0
Dependents          0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
Churn               0
dtype: int64

In [12]:
train, test = train_test_split(df, random_state=42)
train.shape, test.shape

((5282, 20), (1761, 20))

In [0]:
target = 'Churn'
features = df.columns[:-1].tolist()

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

In [0]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
model = Sequential([
    Dense(10, input_shape=(19,), activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(
    x = X_train_scaled,
    y = y_train,
    validation_data=(X_test_scaled, y_test),
    epochs = 20
)

Epoch 1/20
166/166 [==============================] - 0s 3ms/step - loss: 0.5607 - accuracy: 0.7079 - val_loss: 0.4852 - val_accuracy: 0.7677
Epoch 2/20
166/166 [==============================] - 0s 2ms/step - loss: 0.4702 - accuracy: 0.7747 - val_loss: 0.4474 - val_accuracy: 0.7825
Epoch 3/20
166/166 [==============================] - 0s 2ms/step - loss: 0.4494 - accuracy: 0.7806 - val_loss: 0.4333 - val_accuracy: 0.7893
Epoch 4/20
166/166 [==============================] - 0s 2ms/step - loss: 0.4397 - accuracy: 0.7861 - val_loss: 0.4260 - val_accuracy: 0.7990
Epoch 5/20
166/166 [==============================] - 0s 2ms/step - loss: 0.4342 - accuracy: 0.7887 - val_loss: 0.4222 - val_accuracy: 0.8058
Epoch 6/20
166/166 [==============================] - 0s 2ms/step - loss: 0.4293 - accuracy: 0.7940 - val_loss: 0.4188 - val_accuracy: 0.8018
Epoch 7/20
166/166 [==============================] - 0s 2ms/step - loss: 0.4260 - accuracy: 0.7965 - val_loss: 0.4169 - val_accuracy: 0.8064
Epoch 

In [16]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(10, input_shape=(19,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [4,8,16,32,64,128],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7955303430557251 using {'batch_size': 4, 'epochs': 20}
Means: 0.7955303430557251, Stdev: 0.011952029449666826 with: {'batch_size': 4, 'epochs': 20}
Means: 0.7930682182312012, Stdev: 0.009791769014870732 with: {'batch_size': 8, 'epochs': 20}
Means: 0.794014835357666, Stdev: 0.01569493332624257 with: {'batch_size': 16, 'epochs': 20}
Means: 0.7943928956985473, Stdev: 0.010760102061211587 with: {'batch_size': 32, 'epochs': 20}
Means: 0.7947731256484986, Stdev: 0.009601658557643222 with: {'batch_size': 64, 'epochs': 20}
Means: 0.785494613647461, Stdev: 0.012275229303749947 with: {'batch_size': 128, 'epochs': 20}


In [20]:
def create_model(opt='adam'):
    # create model
    model = Sequential()
    model.add(Dense(10, input_shape=(19,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [4],
              'epochs': [20],
              'opt': ['sgd', 'adam', 'nadam', 'rmsprop']}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.801399028301239 using {'batch_size': 4, 'epochs': 20, 'opt': 'sgd'}
Means: 0.801399028301239, Stdev: 0.010323767047500642 with: {'batch_size': 4, 'epochs': 20, 'opt': 'sgd'}
Means: 0.7934455633163452, Stdev: 0.01854383967614564 with: {'batch_size': 4, 'epochs': 20, 'opt': 'adam'}
Means: 0.7955301761627197, Stdev: 0.012975028894269037 with: {'batch_size': 4, 'epochs': 20, 'opt': 'nadam'}
Means: 0.789660918712616, Stdev: 0.014511108082287269 with: {'batch_size': 4, 'epochs': 20, 'opt': 'rmsprop'}


In [28]:
def create_model(lr=0.1):
    # create model
    opt = SGD(learning_rate=lr)
    model = Sequential()
    model.add(Dense(10, input_shape=(19,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [4],
              'lr': [0.001, 0.01, 0.1, 0.2, 0.3, 0.4],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7934468269348145 using {'batch_size': 4, 'epochs': 20, 'lr': 0.01}
Means: 0.7871986150741577, Stdev: 0.013664446920015104 with: {'batch_size': 4, 'epochs': 20, 'lr': 0.001}
Means: 0.7934468269348145, Stdev: 0.010749007460755484 with: {'batch_size': 4, 'epochs': 20, 'lr': 0.01}
Means: 0.7902294635772705, Stdev: 0.01667695217604794 with: {'batch_size': 4, 'epochs': 20, 'lr': 0.1}
Means: 0.7924989581108093, Stdev: 0.017812195907457272 with: {'batch_size': 4, 'epochs': 20, 'lr': 0.2}
Means: 0.7862523674964905, Stdev: 0.011603692343674133 with: {'batch_size': 4, 'epochs': 20, 'lr': 0.3}
Means: 0.7747086644172668, Stdev: 0.017834372643016783 with: {'batch_size': 4, 'epochs': 20, 'lr': 0.4}


In [29]:
def create_model(act='relu'):
    # create model
    opt = SGD(learning_rate=0.01)
    model = Sequential()
    model.add(Dense(10, input_shape=(19,), activation=act))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [4],
              'epochs': [20],
              'act': ['sigmoid', 'elu', 'relu', 'selu', 'softplus', 'tanh']}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7995041966438293 using {'act': 'softplus', 'batch_size': 4, 'epochs': 20}
Means: 0.7955301523208618, Stdev: 0.012152960880097595 with: {'act': 'sigmoid', 'batch_size': 4, 'epochs': 20}
Means: 0.7959078669548034, Stdev: 0.0137782372686079 with: {'act': 'elu', 'batch_size': 4, 'epochs': 20}
Means: 0.7943936228752136, Stdev: 0.009910978699329137 with: {'act': 'relu', 'batch_size': 4, 'epochs': 20}
Means: 0.7907949686050415, Stdev: 0.01331036386726436 with: {'act': 'selu', 'batch_size': 4, 'epochs': 20}
Means: 0.7995041966438293, Stdev: 0.014119668927630207 with: {'act': 'softplus', 'batch_size': 4, 'epochs': 20}
Means: 0.7928795456886292, Stdev: 0.011831740673660166 with: {'act': 'tanh', 'batch_size': 4, 'epochs': 20}


In [32]:
def create_model(act='relu'):
    # create model
    opt = SGD(learning_rate=.01)
    model = Sequential()
    model.add(Dense(10, input_shape=(19,), activation=act))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [4],
              'epochs': [20, 40, 60, 80, 100],
              'act': ['softplus']}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.8010206103324891 using {'act': 'softplus', 'batch_size': 4, 'epochs': 20}
Means: 0.8010206103324891, Stdev: 0.012288085489814884 with: {'act': 'softplus', 'batch_size': 4, 'epochs': 20}
Means: 0.798936378955841, Stdev: 0.014653504049676882 with: {'act': 'softplus', 'batch_size': 4, 'epochs': 40}
Means: 0.7964767694473267, Stdev: 0.012978463857152312 with: {'act': 'softplus', 'batch_size': 4, 'epochs': 60}
Means: 0.7930660605430603, Stdev: 0.013962157923044874 with: {'act': 'softplus', 'batch_size': 4, 'epochs': 80}
Means: 0.7923108220100403, Stdev: 0.014201036438348184 with: {'act': 'softplus', 'batch_size': 4, 'epochs': 100}


I can hypertune the other paramas but this is taking a long time already

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?